In [15]:
import os

Reading API Keys

In [11]:
#Reading OpenAI Key
OPENAI_API_KEY=input("Enter OpenAI API Key")
#Reading Langchain API Key
LANGCHAIN_API_KEY=input("Enter Langchain API Key")

In [12]:
#Enabiling Langchain Tracing
LANGCHAIN_TRACING_V2= "true"
#OpenAI Model
OPENAI_MODEL="gpt-4o-mini"
LANGCHAIN_TRACING_V2="true"


In [ ]:
# installing langchain_openai package
! pip install langchain_openai

In [13]:
# initializing llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model=OPENAI_MODEL,openai_api_key=OPENAI_API_KEY,temperature=0.5)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10ed073b0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10ed07a40>, root_client=<openai.OpenAI object at 0x10ed38470>, root_async_client=<openai.AsyncOpenAI object at 0x10ed07380>, model_name='gpt-4o-mini', temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')

In [ ]:
# Installing Python package for SQLDatabase
! pip install langchain-community==0.2.9
! pip install sqlalchemy==2.0.28

In [14]:
#initializing Database object
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///baggage_tracking.db")
db

In [15]:
from langchain_core.output_parsers import BaseGenerationOutputParser
from typing import List
from langchain_core.exceptions import OutputParserException
from langchain_core.outputs import  Generation
class SQlExtractParser(BaseGenerationOutputParser[str]):
    """
    A parser for SQL output.
    """

    def parse_result(self, result: List[Generation], *, partial: bool = False) -> str:
        """
        Parse the SQL query input and return the extracted SQL query.

        Args:
            sql_query_input (str): The input SQL query string.
            partial (bool, optional): Whether the input is a partial query. Defaults to False.

        Returns:
            str: The extracted SQL query.
        """
        if not result:
            raise OutputParserException("Input SQL query is empty")
        
        print(f"Incoming query {result[0]}")

        if 'SQLQuery:' in result[0].text:
            sql_query = result[0].text.split('SQLQuery:', 1)[1].strip()
            print(f"Extracted SQL query: {sql_query}")
        else:
            sql_query = result[0].text

        return sql_query

In [16]:
sql_examples = [
    {
        "input": "Show bag tag numbers for the PNR number AAA100",
        "query": "select bag_tag_number from baggage_details where pnr = 'AAA100';"
    },
    {
        "input": "Find the baggage details of the flight originating from JFK and destination is SFO",
        "query": "select * from baggage_details where origin ='JFK' and destination ='SFO';"
    },
    {
        "input": "Find scanning details of bag tag number 001234567890",
        "query": "SELECT * from baggage_scanning where bag_tag_number = '001234567890';"
    },
    {
        "input": "Find the passengers in flight BC101",
        "query": "SELECT * from passenger_details where passenger_flight_number = 'BC101';"
    },
    {
        "input": "Find the passenger details for the bag tag number 001234567890",
        "query": "SELECT * from passenger_details where bag_tag_number = '001234567890'"
    },
    {
        "input": "Find rush bags in flight AB100",
        "query": "SELECT * from baggage_details where rush_bag='Yes' and flight_number = 'AB100'"
    }
]

Langchain Example  Selector
=====================================
langchain_core.example_selectors is a module in LangChain that deals with selecting examples dynamically from a set of predefined examples based on the current input or prompt. This is useful when you want to choose the most relevant examples to show or use for few-shot learning, helping to improve the performance of large language models (LLMs) by providing them with more specific and contextual examples.

Example Selectors dynamically pick the most relevant examples from a set, based on the input to the language model. Instead of hardcoding examples for few-shot learning, example selectors allow you to choose relevant examples based on input.



In [ ]:
! pip install langchain_chroma

In [17]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
# initializes an instance of a Chroma vector store.
vector_store = Chroma()
# deletes any existing collection in the vector store
vector_store.delete_collection()
#  LangChain class that helps you select examples based on their semantic similarity to a given input. 
# It uses vector embeddings (vectorized representations of text) to compare the input with examples.
# creating creates an instance of SemanticSimilarityExampleSelector
sql_example_selector = SemanticSimilarityExampleSelector.from_examples(
    sql_examples,
    # To transform SQL examples and input queries into embeddings (vectors), enabling semantic comparison between them.
    OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    # This is the Chroma vector store, where embeddings of the examples will be stored and queried. 
    vector_store,
    # This specifies that the selector should return the top 2 most similar examples. 
    k=2,
    #  This indicates that the key "input" in the provided examples will be used as the query. 
    # It tells the selector what part of each example should be compared to the input query.
    input_keys=["input"]
)

In [18]:
from langchain_core.prompts import ChatPromptTemplate
sql_example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai","{query}")
    ]
)

In [19]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate
sql_few_shot_prompt = FewShotChatMessagePromptTemplate(
    # This is the format/template used for each individual example in the few-shot learning setup. 
    # It defines how each example (SQL query and its corresponding result) will be presented.
    example_prompt=sql_example_prompt,
    # example selector that dynamically chooses the most relevant examples based on the user's input.
    example_selector=sql_example_selector,
    # This indicates which variables (placeholders) from the input will be used in the prompt template.
    input_variables=["input"]
)

In [20]:
SQL_QUERY_PROMPT=ChatPromptTemplate.from_messages(
    [
        ("system",""" You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
        Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
        Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
        Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
        Pay attention to use date('now') function to get the current date, if the question involves "today".

        Use the following format:

        Question: Question here
        SQLQuery: SQL Query to run
        SQLResult: Result of the SQLQuery
         
        {top_k}

        Only use the following tables:
        {table_info}"""),
        sql_few_shot_prompt,
        ("human","{input}")
    ]
)

In [21]:
from langchain_core.prompts import PromptTemplate
ANSWER_PROMPTS = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.
     Combine the the Question and SQL Result to generate a meaningful answer to the user.
     The answer should be in the format of a sentence.
     The answer should be a direct response to the user question.
     Respond to the question as an experienced airline ground staff

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )

Dynamic Table Selection
========================

In [22]:
from langchain_core.prompts import ChatPromptTemplate
TABLE_SELECTION_PROMPT=ChatPromptTemplate.from_messages(
    [
        ("system","""Return the names of ALL the SQL tables that MIGHT be relevant to the user question.
        The tables are:
        {table_names}
        Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed.
        Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
        """),
        ("human","{input}")
    ]
)

In [23]:
from langchain_core.output_parsers import BaseGenerationOutputParser
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.outputs import Generation
from typing import List
class TableNameExtractParser(BaseGenerationOutputParser[str]):

    def parse_result(self, result: List[Generation], *, partial: bool = False) -> str:
        """
        Parse the result and return a comma-separated list of table names.

        Args:
            result (List[Generation]): The input list of generations.
            partial (bool, optional): Whether to parse partially. Defaults to False.

        Returns:
            str: A comma-separated list of table names.

        Raises:
            OutputParserException: If the input table list is empty.
        """
        if not result:
            raise OutputParserException("Input Table list is empty")
        
        print("Table list XA %s",result[0])
        output_parser = CommaSeparatedListOutputParser()
        table_list = output_parser.parse_result(result) 

        return table_list

In [24]:
from operator import itemgetter
table_selection_chain={"input":itemgetter("question"),"table_names":itemgetter("table_names")}|TABLE_SELECTION_PROMPT|llm|TableNameExtractParser()

In [ ]:
! pip install pandas

In [ ]:
import pandas as pd 
table_details = pd.read_csv("./table_details.csv")

table_description_array = []
for index, row in table_details.iterrows():
    table_description_array.append("Table Name:" + row['TableName'] + "\n" + "Table Description:" + row['Description'] + "\n\n")

table_description = '\n'.join(table_description_array)
print("Table description ", table_description)

In [33]:
table_selection_chain.invoke({"question":"On an average how many passengers are there in a flight?",
                             "table_names":table_description})

Table list XA %s text='passenger_details' message=AIMessage(content='passenger_details', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 256, 'total_tokens': 259}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-dba2463f-e295-4e63-86da-968ca7469b24-0', usage_metadata={'input_tokens': 256, 'output_tokens': 3, 'total_tokens': 259})


['passenger_details']

In [36]:
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
# Create a query chain
query_chain = create_sql_query_chain(llm, db,prompt=SQL_QUERY_PROMPT) | SQlExtractParser()

# Create an execution chain
execute_query = QuerySQLDataBaseTool(db=db)
execute_chain = itemgetter("query") | execute_query

# Create an answer chain
answer_chain = ANSWER_PROMPTS | llm | StrOutputParser()

# Combine the chains
final_chain = (
    RunnablePassthrough()
    .assign(table_names_to_use=table_selection_chain)
    .assign(query=query_chain)
    .assign(result=execute_chain)
    | answer_chain
)

# Invoke the chain with the user's query
input_data = {
    "question": "On an average how many passengers are there in a flight?",
    "dialect": "sqlite",
    "top_k":2,
    "table_names": table_description,
}
#answer=query_chain.invoke(input_data)
print(final_chain)
answer = final_chain.invoke(input_data)


first=RunnableAssign(mapper={
  table_names_to_use: {
                        input: RunnableLambda(itemgetter('question')),
                        table_names: RunnableLambda(itemgetter('table_names'))
                      }
                      | ChatPromptTemplate(input_variables=['input', 'table_names'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['table_names'], template="Return the names of ALL the SQL tables that MIGHT be relevant to the user question.\n        The tables are:\n        {table_names}\n        Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed.\n        Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`\n        ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
                      | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10ed073b0>, async_client=<o

In [20]:
answer

'On average, there are about 1 to 2 passengers per flight based on the recent data from our flight records.'